# **Solve sudoku with CNN and backtracking algorithm**

### 1. Train Convolutional Neural Network to detect digits using mnist *dataset*

In [0]:
import tensorflow as tf
import pandas as pd

#load the dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#standardize the pixel values from 0-255 to 0-1 - neural nets don't like large values!
train_images = train_images.reshape(60000, 28, 28, 1).astype('float32')/255

#we cannot have a non-binary array to train with, instead we will make a 'dummy' matrix using Pandas
series = pd.Series(train_labels)
train_labels = pd.get_dummies(series)

#build the model
model = tf.keras.models.Sequential()
#add convolutional layers with input that matches our dataset
model.add(tf.keras.layers.Conv2D(256, kernel_size=(3,3), input_shape=(28,28, 1)))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3)))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3)))
model.add(tf.keras.layers.MaxPool2D((2,2)))
#convert from 2D input to 1D vectors
model.add(tf.keras.layers.Flatten())
#finish our model with densely connected layers
model.add(tf.keras.layers.Dense(140, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

#output layer with 10 units (one per each class 0-9)
model.add(tf.keras.layers.Dense(units=10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_images,
          train_labels, 
          epochs=5)

### 2. Load and pre-process sudoku image to get the grid

In [0]:
#@title Install pytesseract 
!pip install pytesseract

In [0]:
import numpy as np
import cv2
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt

#open the image
img = Image.open('1.png').convert('LA')
newsize = (604, 604) 
img = img.resize(newsize) 

plt.figure()
plt.imshow(img) 
plt.show() 

#take only the brightness value from each pixel of the image
array = np.array(img)[:,:,0]

#invert the image (this is how MNIST digits is formatted)
array = 255-array

#this will be the width and length of each sub-image
divisor = array.shape[0]//9
print(array.shape[0], array.shape[1])
puzzle = []
for i in range(9):
    row = []
    for j in range(9):
        #slice image, reshape it to 28x28 (mnist reader size)
        row.append(cv2.resize(array[i*divisor:(i+1)*divisor,
                                    j*divisor:(j+1)*divisor][5:-5, 5:-5], #the n:-n slice removes the borders from each image
                              dsize=(28,28), 
                              interpolation=cv2.INTER_CUBIC))
    puzzle.append(row)

### 3. Construct List of digits using predicted classes [0 for empty boxes]

In [0]:
#create a 9x9 array of 0s (the sudoku solver doesn't use numpy so I won't here)
board = [
    [0 for _ in range(9)] for _ in range(9)
]

for i, row in enumerate(puzzle):
    for j, image in enumerate(row):
        #if the brightness is above 6, then use the model
        if np.mean(image) > 6:
            #this line of code sets the puzzle's value to the model's prediction
            #the preprocessing happens inside the predict call
            board[i][j] = model.predict_classes(image.reshape(1,28,28,1) \
                                                   .astype('float32')/255)[0]

def print_board(bo):
    for i in range(len(bo)):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - - - ")

        for j in range(len(bo[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")

            if j == 8:
                print(bo[i][j])
            else:
                print(str(bo[i][j]) + " ", end="")

print_board(board)

### 4. Solve Sudoku using backtracking algorithm

In [0]:

def solve(bo):
    find = find_empty(bo)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if valid(bo, i, (row, col)):
            bo[row][col] = i

            if solve(bo):
                return True

            bo[row][col] = 0

    return False


def valid(bo, num, pos):
    # Check row
    for i in range(len(bo[0])):
        if bo[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(bo)):
        if bo[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if bo[i][j] == num and (i,j) != pos:
                return False

    return True

def find_empty(bo):
    for i in range(len(bo)):
        for j in range(len(bo[0])):
            if bo[i][j] == 0:
                return (i, j)  # row, col

    return None

solve(board)
print_board(board)